Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Egorova Polina"
COLLABORATORS = ""

---

# I. $LU$ - разложение квадратной матрицы



Рассмотрим наивную реализацию LU - разложения.

Заметим, что мы используем массивы numpy для представления матриц. [Не используйте 'np.matrix'].


In [ ]:
import numpy as np

def diy_lu(a):
    """Создает LU - разложение матрицы `a`.
    
    Наивное LU - разложение: работает столбец за столбцом, накапливает элементарные треугольные матрицы.
    Без выбора главного элемента.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [ ]:
# Теперь сгенерируем матрицу полного ранга и протестируем наивное разложение.
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

In [ ]:
# Настройка вывода чисел с плавающей точкой для большей ясности
np.set_printoptions(precision=3)

In [ ]:
L, u = diy_lu(a)
print(L, "\n")
print(u, "\n")

# Быстрый тест на адекватность: L @ U должна быть равна изначальной матрице с точностью до ошибок округления.
print(a-L@u)

# II. Необходимость выбора главного элемента

Давайте немного подправим матрицу, изменив в ней один элемент:

In [ ]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

Результирующая матрица имеет полный ранг, но наивное LU-разложение не работает.

In [ ]:
np.linalg.matrix_rank(a1)

In [ ]:
l, u = diy_lu(a1)

print(l, u, sep='\n')

### Тест II.1

Для того, чтобы наивное LU - разложение работало необходимо чтобы все лидирующие миноры матрицы были отличны от нуля. Проверьте, выполнено ли это требование для двух матриц a и a1.

In [ ]:
def minor(a):
    ''' Check if all leading minors are non-zero.
    
    Parameters
    ----------
    a : np.array
        2D array representing the square matrix
    
    Returns
    -------
    answer : bool
        True if all leading minors are non-zero
    '''
    def pivot(A):
    n = len(A)
    P = np.eye(n, dtype=float)
    for j in np.arange(n):
        row_num = max(np.arange(j, n), key=lambda i: abs(A[i][j]))
        if j != row_num:
            tmp = np.copy(P[j])
            P[j] = np.copy(P[row_num])
            P[row_num] = tmp
    return P

def lu_decomposition(A):
    n = len(A)                                                                                                                                                                                                                 
    L = np.eye(n)
    U = np.zeros((n,n))
    P = pivot(A)
    PA = P @ A
    for j in range(n):
        for i in np.arange(j+1):
            s1 = sum(U[k][j] * L[i][k] for k in np.arange(i))
            U[i][j] = PA[i][j] - s1                                                                                                                                                               
        for i in np.arange(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in np.arange(j))
            L[i][j] = (PA[i][j] - s2) / U[j][j]

    return (P, L, U)

In [ ]:
assert minor(a)==True
assert minor(a1)==False

### Тест II.2

Модифицируйте алгоритм diy_lu, чтобы осуществлять выбор главного элемента в столбцах. Для контроля выбора можете использовать матрицу перестановок или массив замен.
Напишите функцию, воссоздающую изначальную матрицу из разложения. Протестируйте свой алгоритм на матрицах a и a1.

In [ ]:
def diy_lu_mod(a):
    '''Perform pivoted LU factorization of the input matrix. 
    
    Parameters
    ----------
    a : np.array
        2D array representing a square matrix with float entries
       
    Returns
    -------
    P, L, U : ndarrays
        factors. Here P is a permutation matrix, L is lower triangular
        with unit diagonal elements, and U upper triangular.
    '''
   def pivot(A):
    n = len(A)
    P = np.eye(n, dtype=float)
    for j in np.arange(n):
        row_num = max(np.arange(j, n), key=lambda i: abs(A[i][j]))
        if j != row_num:
            tmp = np.copy(P[j])
            P[j] = np.copy(P[row_num])
            P[row_num] = tmp
    return P

def lu_decomposition(A):
    n = len(A)                                                                                                                                                                                                                 
    L = np.eye(n)
    U = np.zeros((n,n))
    P = pivot(A)
    PA = P @ A
    for j in range(n):
        for i in np.arange(j+1):
            s1 = sum(U[k][j] * L[i][k] for k in np.arange(i))
            U[i][j] = PA[i][j] - s1                                                                                                                                                               
        for i in np.arange(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in np.arange(j))
            L[i][j] = (PA[i][j] - s2) / U[j][j]

    return (P, L, U)


In [ ]:
P, L, U = diy_lu_mod(a)
P1, L1, U1 = diy_lu_mod(a1)

from numpy.testing import assert_allclose
assert_allclose(a, P @ L @ U, atol=1e-10)
assert_allclose(a1, P1 @ L1 @ U1, atol=1e-10 )
